# The Spinning Effective One-Body Initial Condition Solver

### Author: Tyler Knowles

## This module documents the reduced spinning effective one-body initial condition solver as numerically implemented in LALSuite's SEOBNRv3 gravitational waveform approximant.  That is, we follow Section IV A of [Buonanno, Chen, and Damour (2006)](https://arxiv.org/abs/gr-qc/0508067).

### Throughout this module, we will refer to
* [Pan, Buonanno, Buchman et. al. (2010)](https://arxiv.org/abs/0912.3466v2) as PBB2010,
* [Buonanno, Chen, and Damour (2006)](https://arxiv.org/abs/gr-qc/0508067) as BCD2006,
* [Barausse and Buonanno (2010)](https://arxiv.org/abs/0912.3517) as BB2010,
* [Taracchini, et. al. (2012)](https://arxiv.org/abs/1202.0790) as T2012,
* [Damour, et. al. (2009)](https://arxiv.org/abs/0811.2069) as DIN2009, and
* [Pan, et. al. (2014)](https://arxiv.org/abs/1307.6232) as P2014.

<font color='green'>**This module has been validated against the LALSuite SEOBNRv3/SEOBNRv3_opt code that was reviewed and approved for LIGO parameter estimation by the LIGO Scientific Collaboration.**</font>

<a id='intro'></a>

# Introduction: The Physical System of Interest \[Back to [top](#toc)\]
$$\label{intro}$$

Consider two compact objects (e.g. black holes or neutron stars) with masses $m_{1}$, $m_{2}$ (in solar masses) and spin angular momenta ${\bf S}_{1}$, ${\bf S}_{2}$ in a binary system.  The spinning effective one-body ("SEOB") Hamiltonian $H_{\rm real}$ (see [BB2010](https://arxiv.org/abs/0912.3517) Equation (5.69)) describes the dynamics of this system.  We seek initial conditions for nonadiabatic evolutions of such a system, and follow [BCD2006](https://arxiv.org/abs/gr-qc/0508067) Section IV A.

Please note that throughout this notebook we adpot the following conventions:
1. $c = G = 1$ where $c$ is the speed of light in a vacuum and $G$ is Newton's gravitational constant,
1. hatted vectors (e.g. $\hat{\bf L}_{N}$) usually denote scaled or unit vectors, and
1. the initial inclination angle $\iota$ is chosen to be zero.

Choose a spatial coordinate basis $\left\{ {\bf e}_{0}, {\bf e}_{1}, {\bf e}_{2} \right\}$ so that the initial separation vector ${\bf r}$ between the compact objects lies along the ${\bf e}_{0}$-axis.  We start with the following parameters:
1. binary parameters $m_{1}$, $m_{2}$, ${\bf S}_{1}$, and ${\bf S}_{2}$,
1. initial frequency $f$, and
1. SEOB model parameters (constants dependent on $m_{1}$, $m_{2}$).

Our goal is to produce initial dynamical variables
1. ${\bf x} = \left( x, y, z \right)$, and
1. ${\bf p} = \left( p_{x}, p_{y}, p_{z} \right)$.

We include below the physical parameters necessary to compute the initial conditions.  Solving for the initial conditions requires computing $\frac{ \partial H_{\rm real} }{ \partial p_{i} }$ for $i \in \left\{ r, \theta, \phi \right\}$, <font color='red'>which we defer to another module</font>.

<font color='red'>Please note that in [BCD2006](https://arxiv.org/abs/gr-qc/0508067) the initial conditions are solved for given an initial separation; here we use a given initial frequency instead.  The difference is in our approach to solving Equation (4.8).  Our approach also differs from that found in LALSuite's XLALSimIMRSpinEOBInitialConditionsPrec() function (file: LALSimIMRSpinEOBInitialConditionsPrec.c) because we choose our intial coordinate system so that the inclination angle $\iota$ is zero (this simplifies their step one and step five).</font>

Keep in mind that we take tortoise = 0 until the end (see LALSimIMRSpinPrecEOB.c Line 775).

Besides the physical parameters, we also need the [Euler–Mascheroni constant](https://en.wikipedia.org/wiki/Euler%E2%80%93Mascheroni_constant) $\gamma$ and the [geomtrized](https://en.wikipedia.org/wiki/Geometrized_unit_system) solar mass $\mathcal{M}_{\odot}$, both hard-coded in LALSuite with the significant digits shown below.  (The following links are directed to the appropriate LALSuite documentation: [$\gamma$](https://lscsoft.docs.ligo.org/lalsuite/lal/group___l_a_l_constants__h.html#gac6af32574ff8acaeeafe8bf422281e98) and [$\mathcal{M}_{\odot}$](https://lscsoft.docs.ligo.org/lalsuite/lal/group___l_a_l_constants__h.html#gab83f8c705dda3fd0bb2d5f2470bb9cdd).)

In [27]:
#Let's convert to NumPy, since we don't really need an CSE or other symbolic manipulations here
#import sympy as sp
#from mpmath import factorial, power, fac2, mpmathify
#m1,m2,S10,S11,S12,S20,S21,S22,f = sp.symbols("m1 m2 S10 S11 S12 S20 S21 S22 f",real=True)
#dHdpx,dHdpy,dHdpz = sp.symbols("dHdpx dHdpy dHdpz",real=True)
#rho22v2,rho22v3,rho22v4,rho22v5,rho22v6,rho22v6l,rho22v7,rho22v8,rho22v8l,rho22v10,rho22v10l = sp.symbols("rho22v2 rho22v3 rho22v4 rho22v5 rho22v6 rho22v6l rho22v7 rho22v8 rho22v8l rho22v10 rho22v10l",real=True)
#rho21v1,rho21v2,rho21v3,rho21v4,rho21v5,rho21v6,rho21v6l,rho21v7,rho21v7l,rho21v8,rho21v8l,rho21v10,rho21v10l = sp.symbols("rho21v1 rho21v2 rho21v3 rho21v4 rho21v5 rho21v6 rho21v6l rho21v7 rho21v7l rho21v8 rho21v8l rho21v10 rho21v10l",real=True)
#f21v1,f21v3 = sp.symbols("f21v1 f21v3",real=True)
#rho33v2,rho33v3,rho33v4,rho33v5,rho33v6,rho33v6l,rho33v7,rho33v8,rho33v8l = sp.symbols("rho33v2 rho33v3 rho33v4 rho33v5 rho33v6 rho33v6l rho33v7 rho33v8 rho33v8l",real=True)
#f33v3 = sp.symbols("f33v3",real=True)
#rho32v,rho32v2,rho32v3,rho32v4,rho32v5,rho32v6,rho32v6l,rho32v8,rho32v8l = sp.symbols("rho32v rho32v2 rho32v3 rho32v4 rho32v5 rho32v6 rho32v6l rho32v8 rho32v8l",real=True)
#rho31v2,rho31v3,rho31v4,rho31v5,rho31v6,rho31v6l,rho31v7,rho31v8,rho31v8l = sp.symbols("rho31v2 rho31v3 rho31v4 rho31v5 rho31v6 rho31v6l rho31v7 rho31v8 rho31v8l",real=True)
#f31v3 = sp.symbols("f31v3",real=True)
#rho44v2,rho44v3,rho44v4,rho44v5,rho44v6,rho44v6l = sp.symbols("rho44v2 rho44v3 rho44v4 rho44v5 rho44v6 rho44v6l",real=True)
#rho43v,rho43v2,rho43v4,rho43v5,rho43v6,rho43v6l = sp.symbols("rho43v rho43v2 rho43v4 rho43v5 rho43v6 rho43v6l",real=True)
#f43v = sp.symbols("f43v",real=True)
#rho42v2,rho42v3,rho42v4,rho42v5,rho42v6,rho42v6l = sp.symbols("rho42v2 rho42v3 rho42v4 rho42v5 rho42v6 rho42v6l",real=True)
#rho41v,rho41v2,rho41v4,rho41v5,rho41v6,rho41v6l = sp.symbols("rho41v rho41v2 rho41v4 rho41v5 rho41v6 rho41v6l",real=True)
#f41v = sp.symbols("f41v",real=True)
#rho55v2,rho55v3,rho55v4,rho55v5,rho55v6 = sp.symbols("rho55v2 rho55v3 rho55v4 rho55v5 rho55v6",real=True)
#rho54v2,rho54v3,rho54v4 = sp.symbols("rho54v2 rho54v3 rho54v4",real=True)
#rho53v2,rho53v3,rho53v4,rho53v5 = sp.symbols("rho53v2 rho53v3 rho53v4 rho53v5",real=True)
#rho52v2,rho52v3,rho52v4 = sp.symbols("rho52v2,rho52v3,rho52v4",real=True)
#rho51v2,rho51v3,rho51v4,rho51v5 = sp.symbols("rho51v2 rho51v3 rho51v4 rho51v5",real=True)
#rho66v2,rho66v3,rho66v4 = sp.symbols("rho66v2 rho66v3 rho66v4", real=True)
#rho65v2,rho65v3 = sp.symbols("rho65v2 rho65v3",real=True)
#rho64v2,rho64v3,rho64v4 = sp.symbols("rho64v2 rho64v3 rho64v4",real=True)
#rho63v2,rho63v3 = sp.symbols("rho63v2 rho63v3",real=True)
#rho62v2,rho62v3,rho62v4 = sp.symbols("rho62v2 rho62v3 rho62v4",real=True)
#rho61v2,rho61v3 = sp.symbols("rho61v2 rho61v3",real=True)
#rho77v2,rho77v3 = sp.symbols("rho77v2 rho77v3", real=True)
#rho76v2 = sp.symbols("rho76v2",real=True)
#rho75v2,rho75v3 = sp.symbols("rho75v2 rho75v3",real=True)
#rho74v2 = sp.symbols("rho74v2",real=True)
#rho73v2,rho73v3 = sp.symbols("rho73v2 rho73v3",real=True)
#rho72v2 = sp.symbols("rho72v2",real=True)
#rho71v2,rho71v3 = sp.symbols("rho71v2 rho71v3",real=True)
#rho88v2,rho87v2,rho86v2,rho85v2,rho84v2,rho83v2,rho82v2,rho81v2 = sp.symbols("rho88v2 rho87v2 rho86v2 rho85v2 rho84v2 rho83v2 rho82v2 rho81v2",real=True)
#NQC = sp.symbols("NQC",real=True)
import numpy as np
from scipy.optimize import root
from numpy.linalg import norm

#Input variables
m1 = 10.
m2 = 15.
S1 = np.array([0.01, 0.02, -0.03])
S2 = np.array([0.04, -0.05, 0.06])
f = 20.

gamma = 0.577215664901532860606512090082402431
Msol = 4.925491025543575903411922162094833998e-6

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This module is organized as follows, matching the "steps" listed in [BCD2006](https://arxiv.org/abs/gr-qc/0508067):

1. [Step 1:](#step1) Initial Coordinate Choice
  * [Step 1.a:](#massterms) Mass terms
  * [Step 1.b:](#spinterms) Spin terms
  * [Step 1.c:](#ln) Normalized Orbital Angular Momenutm $\hat{\bf L}_{N}$
  * [Step 1.d:](#rhat) Normalized Position $\hat{\bf r}$
  * [Step 1.e:](#vhat) Normalized Velocity $\hat{\bf v}$
  * [Note](#note)
1. [Step 2:](#step2) Compute ${\bf r}$, ${\bf p}_{r}$, ${\bf p}_{\theta}$, and ${\bf p}_{\phi}$
  * [Step 2.a:](#omega) $\omega$
  * [Step 2.b:](#velocity) Initial Velocity $v$
  * [Step 2.c:](#rootfinding) Root finding
1. [Step 3:](#step3) Rotate $\hat{\bf L} \to {\bf e}_{z}$
  * [Step 3.a:](#phat) Normalize ${\bf p}$
  * [Step 3.b:](#lhat) $\hat{\bf L}$
  * [Step 3.c:](#rotate) Rotation matrix
  * [Step 3.d:](#rotaterhat) Rotate $\hat{\bf r}$
  * [Step 3.e:](#rotatevhat) Rotate $\hat{\bf v}$
  * [Step 3.f:](#rotatelnhat) Rotate $\hat{\bf L}_{N}$
  * [Step 3.g:](#rotates1) Rotate ${\bf S}_{1}$
  * [Step 3.h:](#rotates2) Rotate ${\bf S}_{2}$
  * [Step 3.i:](#rotateshat1) Rotate $\hat{\bf S}_{1}$
  * [Step 3.j:](#rotateshat2) Rotate $\hat{\bf S}_{2}$
  * [Step 3.k:](#rotater) Rotate ${\bf r}$
  * [Step 3.l:](#rotatep) Rotate ${\bf p}$
1. [Step 4:](#step4) Compute $\dot{\bf r}$
1. [Step 5:](#step5) Invert the rotation of Step 3
1. [Output](#latex_pdf_output): Output this module to $\LaTeX$-formatted PDF

<a id='step1'></a>

# Step 1: Initial Coordinate Choice \[Back to [top](#toc)\]
$$\label{step1}$$

<a id='massterms'></a>

## Step 1.a: Mass terms \[Back to [top](#toc)\]
$$\label{massterms}$$

We begin by defining the total mass of the system $M$ and the symmetric mass ratio $\eta$:

\begin{align*}
    M &= m_{1} + m_{2} \\
    \eta &= \frac{ m_{1} m_{2} }{ M^{2} }
\end{align*}

In [28]:
M = m1 + m2
Msqinv = 1/M*M
eta = m1*m2*Msqinv

<a id='spinterms'></a>

## Step 1.b: Spin terms \[Back to [top](#toc)\]
$$\label{spinterms}$$

<font color='red'>Can we find a source that says how the basis for the orientation of ${\bf S}_{1}$, ${\bf S}_{2}$ is chosen?  For now I assume it's in the $\hat{\bf r}, \hat{\bf v}, \hat{\bf L}_{N}$-frame, which is suggestd by the fact that LALSuite rotates them using the same rotation matrix as rotates $\hat{\bf r}$, $\hat{\bf v}$, and $\hat{\bf L}_{N}$ in the case $\iota \not= 0$.</font>  Since we assumed $G = c = 1$, we make the spin angular momenta dimensionless via:

\begin{align*}
    \hat{\bf S}_{1} &= \frac{ 1 }{ M^{2} } {\bf S}_{1} \\
    \hat{\bf S}_{2} &= \frac{ 1 }{ M^{2} } {\bf S}_{2}
\end{align*}

See LALSimIMRSpinEOBInitialConditionsPrec.c Lines 768--771.

In [29]:
S1hat = S1*Msqinv
S2hat = S2*Msqinv

<a id='ln'></a>

## Step 1.c: Normalized Orbital Angular Momenutm $\hat{\bf L}_{N}$ \[Back to [top](#toc)\]
$$\label{ln}$$

Since we assume that the initial separation vector ${\bf r}$ between $m_{1}$ and $m_{2}$ lies along the ${\bf e}_{0}$-axis, the initial orbital plane coincides with the ${\bf e}_{0},{\bf e}_{1}$-plane.  Thus the normalized inital orbital angular momentum vector $\hat{\bf L}_{N}$ is given by

\begin{equation*}
    \hat{\bf L}_{N} = \begin{bmatrix} 0 \\ 0 \\ 1 \end{bmatrix}
\end{equation*}

See LALSimIMRSpinEOBInitialConditionsPrec.c Lines 787--789.

In [30]:
LNhat = np.array([0., 0., 1.])

<a id='rhat'></a>

## Step 1.d: Normalized Position $\hat{\bf r}$ \[Back to [top](#toc)\]
$$\label{rhat}$$

We assumed that the initial separation vector ${\bf r}$ lies along the ${\bf e}_{0}$-axis, so the normalized initial separation vector $\hat{\bf r}$ is given by

\begin{equation*}
    \hat{\bf r} = \begin{bmatrix} 0 \\ 0 \\ 1 \end{bmatrix}.
\end{equation*}

See LALSimIMRSpinEOBInitialConditionsPrec.c Lines 801--803.

In [31]:
rhat = np.array([0., 0., 1.])

<a id='vhat'></a>

## Step 1.e: Normalized Velocity $\hat{\bf v}$ \[Back to [top](#toc)\]
$$\label{vhat}$$

Given normalized orbital angular momentum ($\hat{\bf L}_{N}$) and normalized position ($\hat{\bf r}$), the normalized velocity vector ($\hat{\bf v}$) is given by

\begin{equation*}
    \hat{\bf v} = \frac{ \hat{\bf L}_{N} \times \hat{\bf r} }{ \left\lvert \hat{\bf L}_{N} \times \hat{\bf r} \right\rvert }.
\end{equation*}

Given $\hat{\bf L}_{N} = \begin{bmatrix} 0 \\ 0 \\ 1 \end{bmatrix}$ and $\hat{\bf r} = \begin{bmatrix} 1 \\ 0 \\ 0 \end{bmatrix}$ it is clear that $\hat{\bf v} = \begin{bmatrix} 0 \\ 1 \\ 0 \end{bmatrix}$.

See LALSimIMRSpinEOBInitialConditionsPrec.c Lines 807--811.

In [32]:
vhat = np.array([0., 1., 0.])

<a id='note'></a>

## Note \[Back to [top](#toc)\]
$$\label{note}$$

Since we began assuming $\iota = 0$, we do not need to rotate $\hat{\bf r}$, $\hat{\bf v}$, $\hat{\bf L}_{N}$, ${\bf S}_{1}$, ${\bf S}_{2}$, $\hat{\bf S}_{1}$, or $\hat{\bf S}_{2}$ as is done at LALSimIMRSpinEOBInitialConditionsPrec.c Lines 840-847.

<a id='step2'></a>

# Step 2: Compute ${\bf r}$ and ${\bf p}$ in spherical coordinates \[Back to [top](#toc)\]
$$\label{step2}$$

<a id='omega'></a>

## Step 2.a: $\omega$ \[Back to [top](#toc)\]
$$\label{omega}$$

<font color='red'>Is there a paper reference for this formula?  It's not quite Newtonian ($\omega = 2 \pi f)$.</font>
 
\begin{equation*}
    \omega = M \mathcal{M}_{\odot} \pi f.
\end{equation*}

See LALSimIMRSpinEOBInitialConditionsPrec.c Line 893.

In [79]:
omega = M*Msol*np.pi*f

<a id='velocity'></a>

## Step 2.b: Initial Velocity $v$ \[Back to [top](#toc)\]
$$\label{velocity}$$

<font color='red'>Is there a paper reference for this formula?</font>

\begin{equation*}
    v = \sqrt[3]{ \omega }.
\end{equation*}

See LALSimIMRSpinEOBInitialConditionsPrec.c Line 894.

In [34]:
v = np.cbrt(omega)
vsq = v*v

<a id='rootfinding'></a>

## Step 2.c: Root-finding \[Back to [top](#toc)\]
$$\label{rootfinding}$$

We will write components of the momentum vector ${\bf p}$ in spherical coordinates with components ${\bf p}^{r}$, ${\bf p}^{\theta}$, and ${\bf p}^{\phi}$.  In the special case in which we find ourselves, we have ([BCD2006](https://arxiv.org/abs/gr-qc/0508067) Equation (4.7)):

\begin{align*}
    {\bf p}^{\phi} &= 0 \\
    {\bf p}^{\theta} &= \frac{ \pi }{ 2 }.
\end{align*}

From [BCD2006](https://arxiv.org/abs/gr-qc/0508067) Equations (4.8)--(4.9), we seek to solve

\begin{equation*}
    \begin{bmatrix} \frac{ \partial H }{ \partial {\bf r}^{r} } \\ \frac{ \partial H }{ \partial {\bf p}^{\theta} } \\ \frac{ \partial H }{ \partial {\bf p}^{\phi} } - \omega \end{bmatrix} = \begin{bmatrix} 0 \\ 0 \\ 0 \end{bmatrix}.
\end{equation*}

As the Hamiltonian is given in Cartesian coordinates, this requires computing $\frac{ \partial H }{ \partial {\bf r}^{0} }$, $\frac{ \partial H }{ \partial {\bf p}^{1} }$, and $\frac{ \partial H }{ \partial {\bf p}^{2} }$ and then converting to spherical coordinates via

\begin{align*}
    \frac{\partial H}{\partial {\bf r}^{r}} &= \frac{\partial H}{\partial {\bf r}^{0}} - \frac{\frac{\partial H}{\partial {\bf p}^{1}}{\bf p}^{\phi}}{\left({\bf r}^{r}\right)^{2}} + \frac{\frac{\partial H}{\partial {\bf p}^{2}}{\bf p}^{\theta}}{\left({\bf r}^{r}\right)^{2}} \\
    \frac{\partial H}{\partial {\bf p}^{\theta}} &= -\frac{\frac{\partial H}{\partial {\bf p}^{2}}}{{\bf r}^{r}} \\
    \frac{\partial H}{\partial {\bf p}^{\phi}} &= \frac{\frac{\partial H}{\partial {\bf p}^{1}}}{{\bf r}^{r}}.
\end{align*}

(see LALSimIMRSpinEOBInitialConditionsPrec.c Lines 409--411).  As such, the values we seek to find are ${\bf r}^{r}$, ${\bf p}^{\theta}$, and ${\bf p}^{\phi}$.

<font color='red'>LALSuite uses the following initial guesses for the root-finding routine:

\begin{align*}
    {\bf r}^{r} &= \frac{ 1 }{ v^{2} } \\
    {\bf p}^{\phi} &= v \\
    {\bf p}^{\theta} &= 0.2.
\end{align*}

Note: LALSuite scales the initial guesses given to the root-finding routine; see LALSimIMRSpinEOBInitialConditionsPrec.c Line 899.  The root finder seeks to solve

In the end, we should have a cartesian postition vector ${\bf q}$ and momentum vector ${\bf p}$.  It seems that because we stared with $\iota = 0$ that we can use $\hat{\bf r}$ instead of $\hat{\bf q}$?</font>

In [35]:
#We need to import the following values from the derivative routine
dHdx = 1.
dHdp1 = 2.
dHdp2 = 3.

def root_func(F):
    return [ dHdx + (dHdp2*F[2] - dHdp1*F[1])/(F[0]*F[0]), -dHdp2/F[0], dHdp1/F[0] - omega ]

root_guess = [1/(v*v), v, 0.2]

soln = root(root_func, root_guess, args=(), method='hybr', jac=None, tol=None, callback=None)

q = np.array([soln.x[0], 0., 0.])
p = np.array([0., soln.x[1], soln.x[2]])

<a id='step3'></a>

# Step 3: Rotate $\hat{\bf L} \to {\bf e}_{z}$ \[Back to [top](#toc)\]
$$\label{step3}$$

## Note \[Back to [top](#toc)\]


At this point, LALSimIMRSpinEOBInitialConditionsPrec.c normalizes the Cartesian separation and momentum vectors constructed in [Step 2](#step2).  We already have a normalized separation vector $\hat{\bf r}$, so we skip that step.

<a id='phat'></a>

## Step 3.a: Normalize ${\bf p}$ \[Back to [top](#toc)\]
$$\label{phat}$$

Next we normalize the new position vector ${\bf p}$ that we found in [Step 2](#step2):

\begin{equation*}
    \hat{\bf p} = \frac{ {\bf p} }{ \left\lvert {\bf p} \right\rvert}.
\end{equation*}

See LALSimIMRSpinEOBInitialConditionsPrec.c Line 1101.

In [37]:
phat = p/norm(p)

<a id='lhat'></a>

## Step 3.b: $\hat{\bf L}$ \[Back to [top](#toc)\]
$$\label{lhat}$$

We compute the normalized relativistic angular momentum vector $\hat{\bf L}$:

\begin{equation*}
    \hat{\bf L} = \frac{ \hat{\bf r} \times \hat{\bf p} }{ \left\lvert \hat{\bf r} \times \hat{\bf p} \right\rvert }.
\end{equation*}

See LALSimIMRSpinEOBInitialConditionsPrec.c Lines 1098--1100.

In [38]:
Lhat = np.cross(rhat,phat)
Lhat /= norm(Lhat)

<a id='rotate'></a>

## Step 3.c: Rotation matrix \[Back to [top](#toc)\]
$$\label{rotate}$$

The rotation matrix from the $\left\{ \hat{\bf r}, {\bf v}, \hat{\bf L}_{N} \right\}$ frame to the $\left\{ \hat{\bf r}, {\bf p}, \hat{\bf L} \right\}$ frame is given by

\begin{equation*}
    \begin{bmatrix} \hat{\bf r}^{0} & \hat{\bf r}^{1} & \hat{\bf r}^{2} \\
        \hat{\bf p}^{0} & \hat{\bf p}^{1} & \hat{\bf p}^{2} \\
        \hat{\bf L}^{0} & \hat{\bf L}^{1} & \hat{\bf L}^{2}\end{bmatrix}.
\end{equation*}

See LALSimIMRSpinEOBInitialConditionsPrec.c Line 1107.

In [40]:
rotate = np.array([rhat, phat, Lhat])

<a id='rotaterhat'></a>

## Step 3.d: Rotate $\hat{\bf r}$ \[Back to [top](#toc)\]
$$\label{rotatesrhat}$$

We now rotate $\hat{\bf r}$.  We'll use primes to denote the rotated vector.

\begin{equation*}
    \hat{\bf r}^{\prime} = \begin{bmatrix} \hat{\bf r}^{0} & \hat{\bf r}^{1} & \hat{\bf r}^{2} \\
        \hat{\bf p}^{0} & \hat{\bf p}^{1} & \hat{\bf p}^{2} \\
        \hat{\bf L}^{0} & \hat{\bf L}^{1} & \hat{\bf L}^{2}\end{bmatrix}
        \begin{bmatrix} \hat{\bf r}^{0} \\ \hat{\bf r}^{1} \\ \hat{\bf r}^{2} \end{bmatrix}
\end{equation*}

See LALSimIMRSpinEOBInitialConditionsPrec.c Line 1112.

In [59]:
rhatprm = np.dot(rotate,rhat)

<a id='rotatevhat'></a>

## Step 3.e: Rotate $\hat{\bf v}$ \[Back to [top](#toc)\]
$$\label{rotatevhat}$$

We rotate $\hat{\bf v}$.  We'll use primes to denote the rotated vector.

\begin{equation*}
    \hat{\bf v}^{\prime} = \begin{bmatrix} \hat{\bf r}^{0} & \hat{\bf r}^{1} & \hat{\bf r}^{2} \\
        \hat{\bf p}^{0} & \hat{\bf p}^{1} & \hat{\bf p}^{2} \\
        \hat{\bf L}^{0} & \hat{\bf L}^{1} & \hat{\bf L}^{2}\end{bmatrix}
        \begin{bmatrix} \hat{\bf v}^{0} \\ \hat{\bf v}^{1} \\ \hat{\bf v}^{2} \end{bmatrix}
\end{equation*}

See LALSimIMRSpinEOBInitialConditionsPrec.c Line 1113.

In [60]:
vhatprm = np.dot(rotate, vhat)

<a id='rotatelnhat'></a>

## Step 3.f: Rotate $\hat{\bf L}_{N}$ \[Back to [top](#toc)\]
$$\label{rotatelnhat}$$

We rotate $\hat{\bf L}_{N}$.  We'll use primes to denote the rotated vector.

\begin{equation*}
    \hat{\bf L}_{N}^{\prime} = \begin{bmatrix} \hat{\bf r}^{0} & \hat{\bf r}^{1} & \hat{\bf r}^{2} \\
        \hat{\bf p}^{0} & \hat{\bf p}^{1} & \hat{\bf p}^{2} \\
        \hat{\bf L}^{0} & \hat{\bf L}^{1} & \hat{\bf L}^{2}\end{bmatrix}
        \begin{bmatrix} \hat{\bf L}_{N}^{0} \\ \hat{\bf L}_{N}^{1} \\ \hat{\bf L}_{N}^{2} \end{bmatrix}
\end{equation*}

See LALSimIMRSpinEOBInitialConditionsPrec.c Line 1114.

In [61]:
LNhatprm = np.dot(rotate, LNhat)

<a id='rotates1'></a>

## Step 3.g: Rotate ${\bf S}_{1}$ \[Back to [top](#toc)\]
$$\label{rotates1}$$

We rotate ${\bf S}_{1}$.  We'll use primes to denote the rotated vector.

\begin{equation*}
    {\bf S}_{1}^{\prime} = \begin{bmatrix} \hat{\bf r}^{0} & \hat{\bf r}^{1} & \hat{\bf r}^{2} \\
        \hat{\bf p}^{0} & \hat{\bf p}^{1} & \hat{\bf p}^{2} \\
        \hat{\bf L}^{0} & \hat{\bf L}^{1} & \hat{\bf L}^{2}\end{bmatrix}
        \begin{bmatrix} {\bf S}_{1}^{0} \\ {\bf S}_{1}^{1} \\ {\bf S}_{1}^{2} \end{bmatrix}
\end{equation*}

See LALSimIMRSpinEOBInitialConditionsPrec.c Line 1115.

In [62]:
S1prm = np.dot(rotate, S1)

<a id='rotates2'></a>

## Step 3.h: Rotate ${\bf S}_{2}$ \[Back to [top](#toc)\]
$$\label{rotates2}$$

We rotate ${\bf S}_{2}$.  We'll use primes to denote the rotated vector.

\begin{equation*}
    {\bf S}_{2}^{\prime} = \begin{bmatrix} \hat{\bf r}^{0} & \hat{\bf r}^{1} & \hat{\bf r}^{2} \\
        \hat{\bf p}^{0} & \hat{\bf p}^{1} & \hat{\bf p}^{2} \\
        \hat{\bf L}^{0} & \hat{\bf L}^{1} & \hat{\bf L}^{2}\end{bmatrix}
        \begin{bmatrix} {\bf S}_{2}^{0} \\ {\bf S}_{2}^{1} \\ {\bf S}_{2}^{z} \end{bmatrix}
\end{equation*}

See LALSimIMRSpinEOBInitialConditionsPrec.c Line 1116.

In [63]:
S2prm = np.dot(rotate, S2)

<a id='rotates1hat'></a>

## Step 3.i: Rotate $\hat{\bf S}_{1}$ \[Back to [top](#toc)\]
$$\label{rotates1hat}$$

We rotate $\hat{\bf S}_{1}$.  We'll use primes to denote the rotated vector.

\begin{equation*}
    \hat{\bf S}_{1}^{\prime} = \begin{bmatrix} \hat{\bf r}^{0} & \hat{\bf r}^{1} & \hat{\bf r}^{2} \\
        \hat{\bf p}^{0} & \hat{\bf p}^{1} & \hat{\bf p}^{2} \\
        \hat{\bf L}^{0} & \hat{\bf L}^{1} & \hat{\bf L}^{2}\end{bmatrix}
        \begin{bmatrix} \hat{\bf S}_{1}^{0} \\ \hat{\bf S}_{1}^{1} \\ \hat{\bf S}_{1}^{1} \end{bmatrix}
\end{equation*}

See LALSimIMRSpinEOBInitialConditionsPrec.c Line 1117.

In [52]:
S1hatprm = np.dot(rotate, S1hat)

<a id='rotates2hat'></a>

## Step 3.j: Rotate $\hat{\bf S}_{2}$ \[Back to [top](#toc)\]
$$\label{rotates2hat\hat}$$

We rotate $\hat{\bf S}_{2}$.  We'll use primes to denote the rotated vector.

\begin{equation*}
    \hat{\bf S}_{2}^{\prime} = \begin{bmatrix} \hat{\bf r}^{0} & \hat{\bf r}^{1} & \hat{\bf r}^{2} \\
        \hat{\bf p}^{0} & \hat{\bf p}^{1} & \hat{\bf p}^{2} \\
        \hat{\bf L}^{0} & \hat{\bf L}^{1} & \hat{\bf L}^{2}\end{bmatrix}
        \begin{bmatrix} \hat{\bf S}_{2}^{0} \\ \hat{\bf S}_{2}^{1} \\ \hat{\bf S}_{2}^{2} \end{bmatrix}
\end{equation*}

See LALSimIMRSpinEOBInitialConditionsPrec.c Line 1118.

In [53]:
S2hatprm = np.dot(rotate, S2hat)

<a id='rotater'></a>

## Step 3.k: Rotate ${\bf r}$ \[Back to [top](#toc)\]
$$\label{rotater}$$

We rotate ${\bf r} = {\bf q}$.  We'll use primes to denote the rotated vector.

\begin{equation*}
    {\bf r}^{\prime} = \begin{bmatrix} \hat{\bf r}^{0} & \hat{\bf r}^{1} & \hat{\bf r}^{2} \\
        \hat{\bf p}^{0} & \hat{\bf p}^{1} & \hat{\bf p}^{2} \\
        \hat{\bf L}^{0} & \hat{\bf L}^{1} & \hat{\bf L}^{2}\end{bmatrix}
        \begin{bmatrix} {\bf q}^{0} \\ {\bf q}^{1} \\ {\bf q}^{2} \end{bmatrix}
\end{equation*}

See LALSimIMRSpinEOBInitialConditionsPrec.c Line 1119.

In [56]:
rprm = np.dot(rotate, q)

<a id='rotatep'></a>

## Step 3.l: Rotate ${\bf p}$ \[Back to [top](#toc)\]
$$\label{rotatep}$$

We rotate ${\bf p}$.  We'll use primes to denote the rotated vector.

\begin{equation*}
    {\bf p}^{\prime} = \begin{bmatrix} \hat{\bf r}^{0} & \hat{\bf r}^{1} & \hat{\bf r}^{2} \\
        \hat{\bf p}^{0} & \hat{\bf p}^{1} & \hat{\bf p}^{2} \\
        \hat{\bf L}^{0} & \hat{\bf L}^{1} & \hat{\bf L}^{2}\end{bmatrix}
        \begin{bmatrix} {\bf p}^{0} \\ {\bf p}^{1} \\ {\bf p}^{2} \end{bmatrix}
\end{equation*}

See LALSimIMRSpinEOBInitialConditionsPrec.c Line 1120.

In [57]:
pprm = np.dot(rotate, p)

<a id='step4'></a>

# Step 4: Compute $\dot{\bf r}$ \[Back to [top](#toc)\]
$$\label{step4}$$

<a id='carttosph'></a>

## Convert from Cartesian to Spherical Coordinates \[Back to [top](#toc)\]
$$\label{carttosph}$$

We convert position and momentum into spherical coordinates.  In the special case where $\theta = \frac{ \pi }{ 2 }$ and $\phi = 0$, the spherical position vector ${\bf r} = \left( {\bf r}^{r}, {\bf r}^{\theta}, {\bf r}^{\phi} \right)$ is given by

\begin{align*}
    {\bf r}^{r} &= {\bf r}^{0} \\
    {\bf r}^{\theta} &= \frac{ \pi }{ 2 } \\
    {\bf r}^{\phi} &= 0
\end{align*}

and the spherical momentum vector ${\bf p} = \left( {\bf p}^{r}, {\bf p}^{\theta}, {\bf p}^{\phi} \right)$ is given by

\begin{align*}
    {\bf p}^{r} &= {\bf p}^{0} \\
    {\bf p}^{\theta} &= - {\bf r}^{0}{\bf p}^{2} \\
    {\bf p}^{\phi} &= {\bf r}^{0}{\bf p}^{1} \\
\end{align*}

We call a Cartesian to spherical routine at LALSimIMRSpinEOBInitialConditionsPrec.c Line 1139, and the function itself is defined on Lines 243--285.

In [58]:
r = np.array([rprm[0], np.pi/2., 0.])
psph = np.array([pprm[0], -rprm[0]*pprm[2], rprm[0]*pprm[1]])

<a id='secondderiv'></a>

## Second derivatives of $H_{\rm real}$ \[Back to [top](#toc)\]
$$\label{seconderiv}$$

We need to compute $\frac{ \partial H }{ \partial {\bf p}^{\phi} }$, $\frac{ \partial^{2} H_{\rm real} }{ \partial r^{2} }$, and $\frac{ \partial^{2} H_{\rm real} }{ \partial r \partial {\bf p}^{\phi} }$ (<font color='red'>in another module</font>).

<font color='red'>Note: be sure that, following this, we use normalized spins.</font>

In [67]:
#Need to figure out how to compute and then import these
dHdpphi = dHdp1/r[0] - omega
d2Hdr2 = 0.2
d2Hdrdpphi = 0.3

<a id='dEdr'></a>

## $\frac{ \partial E }{ \partial r }$ \[Back to [top](#toc)\]
$$\label{dEdr}$$

We seek to compute $\frac{ \partial H }{\partial r}$, and [BCD2006](https://arxiv.org/abs/gr-qc/0508067) uses the convention $H \equiv E$.  (see [BCD2006](https://arxiv.org/abs/gr-qc/0508067) Equation (3.7)).  From [BCD2006](https://arxiv.org/abs/gr-qc/0508067) Equation Equation (4.14) (noting that this equation applies in spherical coordinates when ${\bf r}$ is directed along the ${\bf e}_{0}$ axis),

\begin{equation*}
    \frac{ \partial E }{ \partial r } = -\frac{ \frac{ \partial H }{ \partial {\bf p}^{\phi} } \frac{ \partial^{2} H }{ \left(\partial {\bf r}^{r} \right)^{2} } }{ \frac{ \partial^{2} H }{ \partial {\bf r}^{r} \partial {\bf p}^{\phi} } }.
\end{equation*}

In [68]:
dEdr = -dHdpphi*d2Hdr2/d2Hdrdpphi

<a id='skerr'></a>

## ${\bf S}_{\rm Kerr}$ \[Back to [top](#toc)\]
$$\label{skerr}$$

From [BB2010](https://arxiv.org/abs/0912.3517) Equations (5.2), (5.63), and (5.67) we have

\begin{equation*}
    {\bf S}_{\rm Kerr} = {\bf S}_{1} + {\bf S}_{2}.
\end{equation*}

Taking the square of [BB2010](https://arxiv.org/abs/0912.3517) Equation (4.9) <font color='red'>(be careful with the factor or $M$...)</font>,

\begin{equation*}
    a^{2} = \frac{ {\bf S}_{Kerr} \cdot {\bf S}_{Kerr} }{ M^{2} }
\end{equation*}

so that

\begin{equation*}
    a = \sqrt{ a^{2} }.
\end{equation*}

In [69]:
SKerr = np.add(S1, S2)
asq = norm(SKerr)*Msqinv
a = np.sqrt(asq)

<a id='sigmastar'></a>

## $\boldsymbol{\sigma}^{*}$ \[Back to [top](#toc)\]
$$\label{sigmastar}$$

From [BB2010](https://arxiv.org/abs/0912.3517) Equation (5.3),

\begin{equation*}
    \boldsymbol{\sigma}^{*} = \frac{ m_{2} }{ m_{1} } {\bf S}_{1} + \frac{ m_{1} }{ m_{2} }{\bf S}_{2}.
\end{equation*}

In [70]:
sigmastar = np.add(m2/m1*S1, m1/m2*S2)

<a id='Hreal'></a>

## $H_{\rm real}$ \[Back to [top](#toc)\]
$$\label{Hreal}$$

We now compute $H_{\rm real}$ (LALSimIMRSpinEOBInitialConditionsPrec.c Line 1217) (<font color='red'>another module</font>).

In [71]:
Hreal = 1.

<a id='polardata'></a>

## Polar data \[Back to [top](#toc)\]
$$\label{polardata}$$

At LALSimIMRSpinEOBInitialConditionsPrec.c Lines 1234--1238, we set the following polar data ${\bf P}$ <font color='red'>(need to find reference for this?)</font>:

\begin{align*}
    {\bf P}^{0} &= {\bf r}^{r} \\
    {\bf P}^{1} &= 0 \\
    {\bf P}^{2} &= {\bf p}^{r} \\
    {\bf P}^{3} &= {\bf p}^{\phi}
\end{align*}

In [75]:
polar = np.array([r[0], 0., p[0], p[1]])

<a id='vphikepler'></a>

## vPhiKepler \[Back to [top](#toc)\]
$$\label{vphikepler}$$

From [T2012](https://arxiv.org/abs/1202.0790) Equation (A2),

\begin{equation*}
    {\rm vPhiKepler} = \frac{ 1 }{ \omega^{2} \left( {\bf r}^{r} \right)^{3} }.
\end{equation*}

See LALSimIMRSpinEOBFactorizedWaveformPrec_v3opt.c Lines 113 and 1271--1315.  Note that SEOBNRv3_opt recalculates $\omega$, but I think the $\omega$ above is based on a circular orbit and therefore the recalcuation is unnecessary.

In [77]:
vPhiKepler = 1./(omega*omega*r[0]*r[0]*r[0])

/home/tyler/jup/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in double_scalars
  """Entry point for launching an IPython kernel.


<a id='rcrossp'></a>

## ${\bf r} \times {\bf p}$ \[Back to [top](#toc)\]
$$\label{rcrossp}$$

We'll use the notation

\begin{equation*}
    {\rm rcrossp} = {\bf r}^{\prime} \times {\bf p}^{\prime}.
\end{equation*}

See LALSimIMRSpinEOBFactorizedWaveformPrec_v3opt.c Lines 170--172.

In [ ]:
rcrossp0 = rprm1*pprm2 - rprm2*pprm1
rcrossp1 = rprm2*pprm0 - rprm0*pprm2
rcrossp2 = rprm0*pprm1 - rprm1*pprm0

<a id='vphi'></a>

## vPhi \[Back to [top](#toc)\]
$$\label{vphi}$$

We'll use the notation

\begin{equation*}
    {\rm vPhi} = \omega {\bf r}^{r} \sqrt[3]{\rm vPhiKepler}.
\end{equation*}

See LALSimIMRSpinEOBFactorizedWaveformPrec_v3opt.c Lines 185 and 190.

In [ ]:
vPhi = omega*rr*sp.cbrt(vPhiKepler)

<a id='sidot'></a>

## ${\bf S}_{i} \cdot {\bf L}$ \[Back to [top](#toc)\]
$$\label{sidotl}$$

We compute ${\bf S}_{1} \cdot {\bf L}$ and ${\bf S}_{2} \cdot {\bf L}$.

See LALSimIMRSpinEOBFactorizedFluxPrec_v3opt.c lines 131--134.

In [ ]:
s1dotL = S1hatprm0*Lhat0 + S1hatprm1*Lhat1 + S1hatprm2*Lhat2
s2dotL = S2hatprm0*Lhat0 + S2hatprm1*Lhat1 + S2hatprm2*Lhat2

<a id='chii'></a>
## $\boldsymbol{\chi}_{\rm S}$, $\boldsymbol{\chi}_{\rm A}$ \[Back to [top](#toc)\]
$$\label{chii}$$

From [P2014](https://arxiv.org/abs/1307.6232) Equations 17, we have

\begin{align*}
    \chi_{\rm S} = \frac{1}{2} \left( {\bf S}_{1} + {\bf S}_{2} \right) \cdot L \\
    \chi_{\rm A} = \frac{1}{2} \left( {\bf S}_{1} - {\bf S}_{2} \right) \cdot L
\end{align*}

In [ ]:
chiS = 0.5*(s1dotL + s2dotL)
chiA = 0.5*(s1dotL - s2dotL)

<a id='mihat'></a>
## $\hat{m}_{i}$ \[Back to [top](#toc)\]
$$\label{mihat}$$

We scale the masses $m_{1}$, $m_{2}$ by total mass.  See LALSimIMREOBNewtonianMultipole.c Lines 540--541.

\begin{align*}
    \hat{m}_{1} = \frac{ m_{1} }{ M } \\
    \hat{m}_{2} = \frac{ m_{2} }{ M } \\
\end{align*}

In [ ]:
mhat1 = m1/M
mhat2 = m2/M

<a id='newtonianmultipole'></a>

## Newtonian multipole \[Back to [top](#toc)\]
$$\label{newtonianmultipole}$$

The Newtonian multipolar waveform is given in [DIN2009](https://arxiv.org/abs/0811.2069) Equation (4).  For a given $(\ell, m)$ we define

\begin{align*}
    \epsilon &= \left( \ell + m \right) {\rm mod } 2 \\
    n &= \left( i m \right)^{\ell} \frac{ 8 \pi }{ \left( 2 \ell + 1 \right)!! } \sqrt{ \frac{ \left( \ell + 1 \right) \left( \ell + 2 \right) }{ \ell \left( \ell - 1 \right) } }
\end{align*}

along with the associated Legendre function evaluated at zero.  See LALSimIMRSpinEOBFactorizedWaveformPrec_v3opt.c Line 206 and LALSimIMREOBNewtonianMultipole.c Lines 205, 210, 290, and 309--506.

In [ ]:
def AssociatedLegendre(l,m):
    if l==1:
        if m==1:
            return -1.
        else:
            print("You used a bad (l,m)")
    if l==2:
        if m==2:
            return 3.
        elif m==1:
            return 0.
        else:
            print("You used a bad (l,m)")
    if l==3:
        if m==3:
            return 15.
        elif m==2:
            return 0.
        elif m==1:
            return 1.5
        else:
            print("You used a bad (l,m)")
    if l==4:
        if m==4:
            return 105.
        elif m==3:
            return 0.
        elif m==2:
            return -7.5
        elif m==1:
            return 0.
        else:
            print("You used a bad (l,m)")
    if l==5:
        if m==5:
            return -945.
        elif m==4:
            return 0.
        elif m==3:
            return 52.5
        elif m==2:
            return 0.
        elif m==1:
            return -1.875
        else:
            print("You used a bad (l,m)")
    if l==6:
        if m==6:
            return 10395.
        elif m==5:
            return 0.
        elif m==4:
            return -472.5
        elif m==3:
            return 0.
        elif m==2:
            return 13.125
        elif m==1:
            return 0.
        else:
            print("You used a bad (l,m)")
    if l==7:
        if m==7:
            return -135135.
        elif m==6:
            return 0.
        elif m==5:
            return 5197.5
        elif m==4:
            return 0.
        elif m==3:
            return -118.125
        elif m==2:
            return 0.
        elif m==1:
            return 2.1875
        else:
            print("You used a bad (l,m)")
    if l==8:
        if m==8:
            return 2027025.
        elif m==7:
            return 0.
        elif m==6:
            return -67567.5
        elif m==5:
            return 0.
        elif m==4:
            return 1299.375
        elif m==3:
            return 0.
        elif m==2:
            return -19.6875
        elif m==1:
            return 0.
        else:
            print("You used a bad (l,m)")
def NewtonianPrefix(m1,m2,l,m,epsilon,eta):
    Mtot = m1 + m2
    m1hat = m1/Mtot
    m2hat = m2/Mtot
    if (m%2)==0:
        sign = 1
    else:
        sign = -1
    lpepm1 = l + epsilon - 1
    if (m1!=m2) or sign==1:
        c = sp.Pow(m2hat,lpepm1) + sign*sp.Pow(m1hat,lpepm1)
    else:
        if l==2 or l==3:
            c = -1.
        elif l==4 or l==5:
            c = -0.5
        else:
            c = 0.
    n = power(m * sp.I, l)
    doubfact = fac2(2*l+1)
    if epsilon==0:
        n *= 8.*sp.pi/doubfact
        n *= sp.sqrt((l+1)*(l+2)/(l*(l-1)))
    elif epsilon==1:
        n = -n
        n *= sp.I*16.*sp.pi/doubfact
        n *= sp.sqrt(((2*l+1)* (l+2) * (l*l - m*m))/((2*l - 1) * (l+1) * l * (l-1)))
    else:
        print("Epsilon must be 0 or 1")
        exit()
    return n*eta*c

<a id='hlmtab'></a>

## hLMTab \[Back to [top](#toc)\]
$$\label{hlmtab}$$

In order to compute flux, we need to build the matrix "hLMTab".  See [T2012](https://arxiv.org/abs/1202.0790) Equation (17) and the Appendix, along with [this private LIGO doc](https://dcc.ligo.org/T1400476).

In [ ]:
for l in range(2, 9):
    for m in range(1, l+1):
        epsilon = (l + m) % 2
        legendre = AssociatedLegendre(l-epsilon,m)*sp.sqrt(2*l+1*factorial(l-m)/4*sp.pi*factorial(l+m))
        #START HERE, KNOWLES
        #Note that LALSimIMREOBNewtonianMultipole.c Line 74 atrributes the
        #Newtonian prefix calculations to https://arxiv.org/abs/1106.1021v2
        prefix = NewtonianPrefix(m1,m2,l,m,epsilon,eta)
        multipole = prefix*legendre*sp.Pow(vPhi*vPhi,(l+epsilon)/2.)
        if ((l+m)%2)==0:
            Slm = (Hreal*Hreal - 1.)/(2.*eta) + 1.
        else:
            Slm = v*pphi
        eulerlog = gamma + sp.log(2.*m*v)
        k = m*omega
        Hrealk = Hreal * k
        Hrealksq4 = 4. * Hrealk*Hrealk
        Hrealk4pi = 4. * sp.pi *Hrealk
        Tlmprefac = sp.sqrt(Hrealk4pi/(1.-sp.exp(-Hrealk4pi)))/factorial(l)
        Tlmprodfac = 1.
        for i in range(1,l+1):
            Tlmprodfac *= Hrealksq4 + (i*i)
        Tlm = Tlmprefac*sp.sqrt(Tlmprodfac)
        auxflm = 0.
        if l==2:
            if m==2:
                rholm = 1 + vsq * (rho22v2 + v*(rho22v3 + v*(rho22v4 + v*(rho22v5 + v*(rho22v6 + rho22v6l*eulerlog
                                + v*(rho22v7 + v*(rho22v8 + rho22v8l*eulerlog + (rho22v10
                                + rho22v10l*eulerlog)*vsq)))))))
            elif m==1:
                rholm = 1. + v * (rho21v1 + v*(rho21v2 + v*(rho21v3 + v*(rho21v4 + v*(rho21v5 + v*(rho21v6
                                + rho21v6l*eulerlog + v*(rho21v7 + rho21v7l*eulerlog + v*(rho21v8 + rho21v8l*eulerlog
                                + (rho21v10 + rho21v10l*eulerlog)*vsq))))))))
                auxflm = v * f21v1 + vsq*v*f21v3
            else:
                print("You used a bad (l,m)")
        elif l==3:
            if m==3:
                rholm = 1. + vsq*(rho33v2 + v*(rho33v3 + v*(rho33v4 + v*(rho33v5 + v*(rho33v6 + rho33v6l*eulerlog
                                + v*(rho33v7 + (rho33v8 + rho33v8l*eulerlog)*v))))))
                auxflm = v*vsq*f33v3;
            elif m==2:
                rholm = 1. + v*(rho32v + v*(rho32v2 + v*(rho32v3 + v*(rho32v4 + v*(rho32v5 + v*(rho32v6
                                + rho32v6l*eulerlog + (rho32v8 + rho32v8l * eulerlog) * vsq))))))
            elif m==1:
                rholm = 1. + vsq*(rho31v2 + v*(rho31v3 + v*(rho31v4 + v*(rho31v5 + v*(rho31v6 + rho31v6l*eulerlog
                                + v*(rho31v7 + (rho31v8 + rho31v8l*eulerlog)*v))))))
                auxflm = v*vsq*f31v3
            else:
                print("You used a bad (l,m)")
        elif l==4:
            if m==4:
                rholm = 1. + vsq*(rho44v2 + v*(rho44v3 + v*(rho44v4 + v*(rho44v5 + (rho44v6 + rho44v6l*eulerlog)*v))))
            elif m==3:
                rholm = 1. + v*(rho43v + v*(rho43v2 + vsq*(rho43v4 + v*(rho43v5 + (rho43v6 + rho43v6l*eulerlog)*v))))
                auxflm = v*f43v
            elif m==2:
                rholm = 1. + vsq*(rho42v2 + v*(rho42v3 + v*(rho42v4 + v*(rho42v5 + (rho42v6 + rho42v6l * eulerlog)*v))))
            elif m==1:
                rholm = 1. + v*(rho41v + v*(rho41v2 + vsq*(rho41v4 + v*(rho41v5 + (rho41v6 + rho41v6l*eulerlog)*v))))
                auxflm = v*f41v
            else:
                print("You used a bad (l,m)")
        elif l==5:
            if m==5:
                rholm = 1. + vsq*(rho55v2 + v*(rho55v3 + v*(rho55v4 + v*(rho55v5 + rho55v6*v))))
            elif m==4:
                rholm = 1. + vsq*(rho54v2 + v*(rho54v3 + rho54v4*v))
            elif m==3:
                rholm = 1. + vsq*(rho53v2 + v*(rho53v3 + v*(rho53v4 + rho53v5*v)))
            elif m==2:
                rholm = 1. + vsq*(rho52v2 + v*(rho52v3 + rho52v4*v))
            elif m==1:
                rholm = 1. + vsq*(rho51v2 + v*(rho51v3 + v*(rho51v4 + rho51v5*v)))
            else:
                print("You used a bad (l,m)")
        elif l==6:
            if m==6:
                rholm = 1. + vsq*(rho66v2 + v*(rho66v3 + rho66v4*v))
            elif m==5:
                rholm = 1. + vsq*(rho65v2 + rho65v3*v)
            elif m==4:
                rholm = 1. + vsq*(rho64v2 + v*(rho64v3 + rho64v4*v))
            elif m==3:
                rholm = 1. + vsq*(rho63v2 + rho63v3*v)
            elif m==2:
                rholm = 1. + vsq*(rho62v2 + v*(rho62v3 + rho62v4*v))
            elif m==1:
                rholm = 1. + vsq*(rho61v2 + rho61v3*v)
            else:
                print("You used a bad (l,m)")
        elif l==7:
            if m==7:
                rholm = 1. + vsq*(rho77v2 + rho77v3*v)
            elif m==6:
                rholm = 1. + rho76v2*vsq
            elif m==5:
                rholm = 1. + vsq*(rho75v2 + rho75v3*v)
            elif m==4:
                rholm = 1. + rho74v2*vsq
            elif m==3:
                rholm = 1. + vsq*(rho73v2 + rho73v3*v)
            elif m==2:
                rholm = 1. + rho72v2*vsq
            elif m==1:
                rholm = 1. + vsq*(rho71v2 + rho71v3*v)
            else:
                print("You used a bad (l,m)")
        elif l==8:
            if m==8:
                rholm = 1. + rho88v2*vsq
            elif m==7:
                rholm = 1. + rho87v2*vsq
            elif m==6:
                rholm = 1. + rho86v2*vsq
            elif m==5:
                rholm = 1. + rho85v2*vsq
            elif m==4:
                rholm = 1. + rho84v2*vsq
            elif m==3:
                rholm = 1. + rho83v2*vsq
            elif m==2:
                rholm = 1. + rho82v2*vsq
            elif m==1:
                rholm = 1. + rho81v2*vsq
            else:
                print("You used a bad (l,m)")
        else:
            print("You used a bad (l,m)")
        rholmPowl = sp.Pow(rholm,l)
        if eta==0.25 and (m % 2):
            rholmPowl = auxflm
        else:
            rholmPowl += auxflm
        #Need to compute NQC corrections! Set as symbol for now.
        hlm = Tlm*Slm*rholmPowl*multipole
        if (m*m*omega*omega*hlm*hlm) > 5.:
            hlm *= NQC
        flux += m*m*omega*omega*hlm*hlm
        if omega*omega > 1 or flux > 5:
            flux = 0.
        flux *= 8./sp.pi
flux /= eta
rdot = -flux/dEdr
pr = rdot/(dHdpr/px)
#convert spherical to cartesian

<a id='step5'></a>

# Step 5: Invert the rotation of Step 3 \[Back to [top](#toc)\]
$$\label{step5}$$

<a id='invrotationmatrix'></a>

## Inverse Rotation Matrix \[Back to [top](#toc)\]
$$\label{invrotationmatrix}$$

The matrix to invert the rotation applied in [Step 3](#step3) is:

\begin{equation*}
    \begin{bmatrix} \hat{\bf r}^{0} & \hat{\bf p}^{0} & \hat{\bf L}^{0} \\
        \hat{\bf r}^{1} & \hat{\bf p}^{1} & \hat{\bf L}^{1} \\
        \hat{\bf r}^{2} & \hat{\bf p}^{2} & \hat{\bf L}^{2}\end{bmatrix}.
\end{equation*}

To see that this is indeed the correct matrix inverse, note that by construction $\hat{\bf q}$, $\hat{\bf p}$, and $\hat{\bf L}$ are all unit vectors orthogonal to one another.  See LALSimIMRSpinEOBInitialConditionsPrec.c Line 1107.

In [ ]:
invert00 = rhat0
invert01 = phat0
invert02 = Lhat0
invert10 = rhat1
invert11 = phat1
invert12 = Lhat1
invert20 = rhat2
invert21 = phat2
invert22 = Lhat2

<a id='invrotaterhat'></a>

## Rotate $\hat{\bf r}^{\prime}$ \[Back to [top](#toc)\]
$$\label{invrotaterhat}$$

We rotate $\hat{\bf r}^{\prime}$ and call the new separation vector ${\bf r}$.

\begin{equation*}
    \hat{\bf r} = \begin{bmatrix} \hat{\bf r}^{0} & \hat{\bf p}^{0} & \hat{\bf L}^{0} \\
        \hat{\bf r}^{1} & \hat{\bf p}^{1} & \hat{\bf L}^{1} \\
        \hat{\bf r}^{2} & \hat{\bf p}^{2} & \hat{\bf L}^{2} \end{bmatrix}
        \begin{bmatrix} \hat{\bf r}^{\prime 0} \\ \hat{\bf r}^{\prime 1} \\ \hat{\bf r}^{\prime 2} \end{bmatrix}
\end{equation*}

See LALSimIMRSpinEOBInitialConditionsPrec.c Line 1315.

In [ ]:
rhat0 = rhat0*rhatprm0 + phat0*rhatprm1 + Lhat0*rhatprm2
rhat1 = rhat1*rhatprm0 + phat1*rhatprm1 + Lhat1*rhatprm2
rhat0 = rhat2*rhatprm0 + phat2*rhatprm1 + Lhat2*rhatprm2

<a id='invrotatevhat'></a>

## Rotate $\hat{\bf v}^{\prime}$ \[Back to [top](#toc)\]
$$\label{invrotatevhat}$$

We rotate $\hat{\bf v}^{\prime}$ and call the new separation vector ${\bf v}$.

\begin{equation*}
    \hat{\bf v} = \begin{bmatrix} \hat{\bf r}^{0} & \hat{\bf p}^{0} & \hat{\bf L}^{0} \\
        \hat{\bf r}^{1} & \hat{\bf p}^{1} & \hat{\bf L}^{1} \\
        \hat{\bf r}^{2} & \hat{\bf p}^{2} & \hat{\bf L}^{2} \end{bmatrix}
        \begin{bmatrix} \hat{\bf v}^{\prime 0} \\ \hat{\bf v}^{\prime 1} \\ \hat{\bf v}^{\prime 2} \end{bmatrix}
\end{equation*}

See LALSimIMRSpinEOBInitialConditionsPrec.c Line 1316.

In [ ]:
vhat0 = rhat0*vhatprm0 + phat0*vhatprm1 + Lhat0*vhatprm2
vhat1 = rhat1*vhatprm0 + phat1*vhatprm1 + Lhat1*vhatprm2
vhat2 = rhat2*vhatprm0 + phat2*vhatprm1 + Lhat2*vhatprm2

<a id='invrotatelnhat'></a>

## Rotate $\hat{\bf L}_{N}^{\prime}$ \[Back to [top](#toc)\]
$$\label{invrotatelnhat}$$

We rotate $\hat{\bf L}_{N}^{\prime}$ and call the new separation vector ${\bf L}_{N}$.

\begin{equation*}
    \hat{\bf L}_{N} = \begin{bmatrix} \hat{\bf r}^{0} & \hat{\bf p}^{0} & \hat{\bf L}^{0} \\
        \hat{\bf r}^{1} & \hat{\bf p}^{1} & \hat{\bf L}^{1} \\
        \hat{\bf r}^{2} & \hat{\bf p}^{2} & \hat{\bf L}^{2} \end{bmatrix}
        \begin{bmatrix} \hat{\bf L}_{N}^{\prime 0} \\ \hat{\bf L}_{N}^{\prime 1} \\ \hat{\bf L}_{N}^{\prime 2} \end{bmatrix}
\end{equation*}

See LALSimIMRSpinEOBInitialConditionsPrec.c Line 1317.

In [ ]:
LNhat0 = rhat0*LNhatprm0 + phat0*LNhatprm1 + Lhat0*LNhatprm2
LNhat1 = rhat1*LNhatprm0 + phat1*LNhatprm1 + Lhat1*LNhatprm2
LNhat2 = rhat2*LNhatprm0 + phat2*LNhatprm1 + Lhat2*LNhatprm2

<a id='tortoise_matrix'></a>

# Tortoise Conversion Matrix \[Back to [top](#toc)\]
$$\label{tortoise_matrix}$$

<font color='red'>We're now back to LALSpinPrecHcapRvecDerivative_v3opt.c, Lines 92--96.</font>

From [Pan, Buonanno, Buchman, et. al. (2010)](https://arxiv.org/abs/0912.3466v2) Equation (A3) the matrix for the coordinate conversion to tortoise coordinates is

\begin{align*}
    \begin{pmatrix} 1 + \frac{ x^{2} }{ r^{2} } \left( \xi - 1 \right) & \frac{ x y }{ r^{2} } \left( \xi - 1 \right) & \frac{ x z }{ r^{2} } \left( \xi - 1 \right) \\
        \frac{ x y }{ r^{2} } \left( \xi - 1 \right) & 1 + \frac{ y^{2} }{ r^{2} } \left( \xi - 1 \right) & \frac{ y z }{ r^{2} } \left( \xi - 1 \right) \\
        \frac{ x z }{ r^{2} } \left( \xi - 1 \right) & \frac{ y z }{ r^{2} } \left( \xi - 1 \right) & 1 + \frac{ z^{2} }{ r^{2} } \left( \xi - 1 \right) \end{pmatrix}
\end{align*}

In [ ]:
ximinus1 = xi - 1
toTort = sp.Array([[1 + x*x*ximinus1/(r*r), x*y*ximinus1/(r*r), x*z*ximinus1/(r*r)],
                   [x*y*ximinus1/(r*r), 1 + y*y*ximinus1/(r*r), y*z*ximinus1/(r*r)],
                   [x*z*ximinus1/(r*r), y*z*ximinus1/(r*r), 1 + z*z*ximinus1/(r*r)]])

<a id='latex_pdf_output'></a>

# Output: Output this module to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

In [ ]:
!jupyter nbconvert --to latex --template latex_nrpy_style.tplx Tutorial-SEOBNR_Initial_Conditions.ipynb
!pdflatex -interaction=batchmode Tutorial-SEOBNR_Initial_Conditions.tex
!pdflatex -interaction=batchmode Tutorial-SEOBNR_Initial_Conditions.tex
!pdflatex -interaction=batchmode Tutorial-SEOBNR_Initial_Conditions.tex
!rm -f Tut*.out Tut*.aux Tut*.log